# NOAA Atmospheric Climate Data Records


### Description
NOAA's Climate Data Records (CDRs) are robust, sustainable, and scientifically sound climate records that provide trustworthy information on how, where, and to what extent the land, oceans, atmosphere and ice sheets are changing. These datasets are thoroughly vetted time series measurements with the longevity, consistency, and continuity to assess and measure climate variability and change. NOAA CDRs are vetted using standards established by the National Research Council (NRC).

Climate Data Records are created by merging data from surface, atmosphere, and space-based systems across decades. NOAA’s Climate Data Records provides authoritative and traceable long-term climate records. NOAA developed CDRs by applying modern data analysis methods to historical global satellite data. This process can clarify the underlying climate trends within the data and allows researchers and other users to identify economic and scientific value in these records. NCEI maintains and extends CDRs by applying the same methods to present-day and future satellite measurements.

Atmospheric Climate Data Records are measurements of several global variables to help characterize the atmosphere at or just above the land and ocean surface as well as other upper air composition variables.

ref: https://registry.opendata.aws/noaa-cdr-atmospheric/

In [9]:
import httpx
import json

endpoint = "https://t4z3zffpeh.execute-api.us-east-1.amazonaws.com/md"

#### Variables

Before we can `visualize` the dataset we HAVE TO get the list of available variables within the NetCDF file. For this we can use the `/variables` endpoint.

In [5]:
resp = httpx.get(f"{endpoint}/variables", params={"url": "s3://noaa-cdr-precip-gpcp-daily-pds/data/2023/gpcp_v01r03_daily_d20231231_c20240308.nc"})
print(resp.json())

['precip']


### Dataset Info

In [10]:
resp = httpx.get(
    f"{endpoint}/info",
    params={
        "url": "s3://noaa-cdr-precip-gpcp-daily-pds/data/2023/gpcp_v01r03_daily_d20231231_c20240308.nc",
        "variable": "precip"
    }
)
print(json.dumps(resp.json(), indent=4))

{
    "bounds": [
        -180.5,
        -90.5,
        179.5,
        89.5
    ],
    "crs": "http://www.opengis.net/def/crs/EPSG/0/4326",
    "band_metadata": [
        [
            "b1",
            {}
        ]
    ],
    "band_descriptions": [
        [
            "b1",
            "value"
        ]
    ],
    "dtype": "float32",
    "nodata_type": "None",
    "name": "precip",
    "count": 1,
    "width": 360,
    "height": 180,
    "attrs": {
        "long_name": "NOAA Climate Data Record (CDR) of Daily GPCP Satellite-Gauge Combined Precipitation",
        "standard_name": "lwe_precipitation_rate",
        "units": "mm/day",
        "valid_range": [
            0.0,
            100.0
        ],
        "cell_methods": "area: mean time: mean",
        "valid_min": 0.0,
        "valid_max": 100.0
    }
}


### Map

In [13]:
from folium import Map, TileLayer

r = httpx.get(
    f"{endpoint}/WebMercatorQuad/tilejson.json",
    params = {
        "url": "s3://noaa-cdr-precip-gpcp-daily-pds/data/2023/gpcp_v01r03_daily_d20231231_c20240308.nc",
        "variable": "precip",
        "rescale": "0,50",
        "colormap_name": "viridis",
    }
).json()

bounds = r["bounds"]
minzoom = r["minzoom"]

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=minzoom
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Noaa"
).add_to(m)

m